In [1]:
import sys
sys.path.insert(0, '..')

from pathlib import Path
import pandas as pd
from tqdm import tqdm
import copy
from glob import glob
import json
from itertools import groupby

In [2]:
# utility to prepare the true timers for mode encoding
class PrepareTrueModeTimers:
    def __init__(self, data_path):
        self._categories = {"ZW_CTRL.MODE": [1, 2, 3, 4, 5, 13, 31, 32, 33, 45, 61, 83]}
        self._oh_only = True
        self._mode_col = "Program:ZW.CTRL_P.STEP"
        self._one_hot_mode_cols = [f"ZW_CTRL.MODE${mode}" for mode in self._categories["ZW_CTRL.MODE"]]
        self.DATA_PATHS = [x for x in glob(str(data_path) + "/*.log.gz")]
        self.DATA_PATHS.sort()
    
    def read_csv(self, path):
        df = pd.read_csv(path, skiprows=[1])
        for c in df.columns:
            try:
                df[c] = df[c].astype(float)
            except:
                pass
        return df
    
    def mode_transform(self, x):
        if self._mode_col in x:
            x["ZW_CTRL.MODE"] = x[self._mode_col] // 100
            x["ZW_CTRL._STEP"] = (x[self._mode_col] % 100) / 100  # normalize to [0, 1]
            del x[self._mode_col]
        return x
    
    def one_hot_encode(self, x):
        out = copy.deepcopy(x)
        for col in self._categories:
            for cat in self._categories[col]:
                out[col + "$" + str(cat)] = int(x[col] == cat)
            if self._oh_only:
                del out[col]
        return out

    def get_true_mode_times(self, mode_data):
        # Implementation inspired by https://codereview.stackexchange.com/questions/239213/find-start-stop-indices-where-array-is-true
        start = [i for i, (x1, x2) in enumerate(zip([0] + mode_data[:-1], mode_data)) if x2 > x1]
        stop = [i for i, (x1, x2) in enumerate(zip(mode_data, mode_data[1:] + [0])) if x2 < x1]
        assert len(start) == len(stop)
        return [[start[i], stop[i], stop[i] - start[i] + 1] for i in range(len(start))]

    # Alternate implementation
    # def get_true_mode_times_v2(self, mode_data):
    #     # Implementation inspired by https://codereview.stackexchange.com/questions/239213/find-start-stop-indices-where-array-is-true
    #     return [[group[0][0], group[-1][0], group[-1][0] - group[0][0] + 1] for group in
    #             [list(group) for key, group in groupby(enumerate(mode_data), key=lambda x: x[1]) if key == 1]
    #         ]

    def get_one_hot_mode_data(self, data):
        mode_data = data[[self._mode_col]]
        obs_dict = mode_data.to_dict("records")
        oh_mode_data = []
        for obs in tqdm(obs_dict):
            obs_p = self.mode_transform(obs)
            obs_p = self.one_hot_encode(obs_p)
            oh_mode_data.append(obs_p)
        return pd.DataFrame(oh_mode_data)

    def process(self):
        out = {}
        for fp in self.DATA_PATHS:
            # Pre-process the data
            name = fp.split('/')[-1]
            print(f"Processing fp => {name}")
            data = self.read_csv(fp)
            modes_data = self.get_one_hot_mode_data(data)

            # Now get true mode times
            d = {}
            for mode_col in self._one_hot_mode_cols:
                key = mode_col.split('$')[-1]
                mode_data = modes_data[mode_col].to_list()
                mode_stats = self.get_true_mode_times(mode_data)
                d[key] = mode_stats
            out[name] = d
        return out

In [5]:
OUT_PATH = Path("/home/haseebs/workspace/predictions-for-rl/data-prep/computed_ranges/4months/true_mode.json")

with open(OUT_PATH, "w") as f:
    json.dump(out, f)

In [3]:
DATA_PATH = Path("../data-logs/logs")
OUT_PATH = Path("/home/haseebs/workspace/predictions-for-rl/data-prep/computed_ranges/4months")

pptm = PrepareTrueModeTimers(data_path=DATA_PATH)
out = pptm.process()
with open(OUT_PATH, "w") as f:
    json.dump(out, f)

Processing fp => observations_2022-11-14T010732Z.log.gz


100%|██████████| 58327/58327 [00:00<00:00, 168755.76it/s]


Processing fp => observations_2022-11-14T171940Z.log.gz


100%|██████████| 58296/58296 [00:00<00:00, 176760.97it/s]


Processing fp => observations_2022-11-15T093117Z.log.gz


...

Processing fp => observations_2023-02-13T113443Z.log.gz


100%|██████████| 58120/58120 [00:00<00:00, 181766.69it/s]


Processing fp => observations_2023-02-14T034324Z.log.gz


100%|██████████| 58307/58307 [00:00<00:00, 178487.47it/s]


IsADirectoryError: [Errno 21] Is a directory: '/home/haseebs/workspace/predictions-for-rl/data-prep/computed_ranges/4months'

In [4]:
def checkConsecutive(l):
    return sorted(l) == list(range(min(l), max(l)+1))

# Check that all timesteps are accounted for exactly once in the extracted mode ranges
for log_file, mode_dic in tqdm(out.items()):
    print("Checking output for log file: ", log_file)
    timesteps = []
    for mode, mode_ranges in mode_dic.items():
        for mode_range in mode_ranges:
            timesteps.extend(list(range(mode_range[0], mode_range[1]+1)))
    assert(checkConsecutive(timesteps))

    # Check that the number of timesteps is the same as the number of rows in the data
    # TODO: could process this much faster if we get the number of rows without re-reading the log file
    assert(sorted(timesteps)[0] == 0)
    data = pd.read_csv(DATA_PATH / log_file, skiprows=[1])
    assert(len(timesteps) == len(data))

print("all good")


  0%|          | 0/137 [00:00<?, ?it/s]

Checking output for log file:  observations_2022-11-14T010732Z.log.gz


  1%|          | 1/137 [00:01<03:08,  1.38s/it]

Checking output for log file:  observations_2022-11-14T171940Z.log.gz


  1%|▏         | 2/137 [00:02<03:01,  1.34s/it]

...

Checking output for log file:  observations_2023-02-13T113443Z.log.gz


 99%|█████████▉| 136/137 [02:57<00:01,  1.29s/it]

Checking output for log file:  observations_2023-02-14T034324Z.log.gz


100%|██████████| 137/137 [02:58<00:00,  1.30s/it]

all good
